<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/achatbot_daily_month_narration_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi && lscpu

Thu Oct 31 05:33:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   63C    P8              14W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!cd /content && rm -rf achatbot && git clone --recursive https://github.com/ai-bot-pro/achatbot.git

In [3]:
%cd /content/achatbot


/content/achatbot


In [ ]:
!bash scripts/pypi_achatbot.sh dev


In [15]:
!pip install -q "dist/achatbot-0.0.7.2-py3-none-any.whl[daily_room_audio_stream,deep_translator,openai,google_ai,together_ai,diffusers,bitsandbytes,tts_edge,queue]"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.5 MB/s eta 0:00:00


In [16]:
!pip show torch torchaudio torchvision diffusers bitsandbytes deep_translator

Name: torch
Version: 2.4.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-nccl-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, bitsandbytes, fastai, peft, sentence-transformers, timm, torchaudio, torchvision
---
Name: torchaudio
Version: 2.4.1
Summary: An audio package for PyTorch
Home-page: https://github.com/pytorch/audio
Author: Soumith Chintala, David Pollack, Sean Naren, Peter Goldsborough, Moto Hira, Caroline Chen, Jeff Hwang, Zhaoheng Ni, Xiaohui Zhang
Author-email: soumith@pytorch.org
License: 
Location: /usr/loca

run bot task woker with bot.json, e.g.: dummy_bot.json

- use daily/livekit room stream, u can click bot joined the room url, to start chat with bot with audio and camera stream,
  - [daily](https://www.daily.co/) need DAILY_API_KEY
- use openai/groq/together.ai api llm model need api key
  - [openai](https://openai.com/) OPENAI_API_KEY
  - [groq](https://groq.com/) GROQ_API_KEY
  - [together.ai](https://www.together.ai/) TOGETHER_API_KEY
- use image gen model:
  - [openai](https://openai.com/) OPENAI_API_KEY
  - [together.ai](https://www.together.ai/) TOGETHER_API_KEY
  - [huggingface api](https://huggingface.co/docs/api-inference/tasks/text-to-image) HF_API_KEY
  - [HFStableDiffusion](https://huggingface.co/docs/diffusers/using-diffusers/conditional_image_generation) HF_TOKEN/HF_API_KEY




In [7]:
from google.colab import userdata
DAILY_API_KEY=userdata.get('DAILY_API_KEY')
TOGETHER_API_KEY=userdata.get('TOGETHER_API_KEY')
HF_TOKEN=userdata.get('HF_TOKEN')


In [ ]:
!huggingface-cli login --token $HF_TOKEN --add-to-git-credential

In [ ]:
!huggingface-cli download stabilityai/stable-diffusion-3.5-large  --local-dir ./models/stabilityai/stable-diffusion-3.5-large --local-dir-use-symlinks False


In [ ]:
!mkdir -p images
!IMAGE_GEN_PROCESSOR=HFStableDiffusionImageGenProcessor DEVICE=auto SD_MODEL=./models/stabilityai/stable-diffusion-3.5-large \
    python -m unittest test.integration.processors.test_image_gen_processor.TestProcessor

In [12]:
!cat /content/daily_month_narration_bot.json

{
  "chat_bot_name": "DailyMonthNarrationBot",
  "room_name": "chat-room",
  "room_url": "",
  "token": "",
  "room_manager": {
    "tag": "daily_room",
    "args": {
      "privacy": "public"
    }
  },
  "services": {
    "pipeline": "achatbot",
    "llm": "together",
    "img-gen": "together",
    "tts": "edge"
  },
  "config": {
    "llm": {
      "tag": "openai_llm_processor",
      "base_url": "https://api.together.xyz/v1",
      "model": "Qwen/Qwen2-72B-Instruct",
      "language": "en",
      "messages": [{}]
    },
    "img_gen": {
      "tag": "TogetherImageGenProcessor",
      "args": {
        "gen_rate_s": 10,
        "model": "black-forest-labs/FLUX.1-schnell-Free"
      }
    },
    "tts": {
      "tag": "tts_edge",
      "args": {
        "voice_name": "zh-CN-YunjianNeural",
        "language": "zh",
        "gender": "Male"
      }
    }
  },
  "config_list": []
}


In [17]:
!DAILY_API_KEY=$DAILY_API_KEY TOGETHER_API_KEY=$TOGETHER_API_KEY \
  python -m src.cmd.bots.main -f /content/daily_month_narration_bot.json

{
    "chat_bot_name": "DailyMonthNarrationBot",
    "config": {
        "img_gen": {
            "args": {
                "gen_rate_s": 10,
                "model": "black-forest-labs/FLUX.1-schnell-Free"
            },
            "tag": "TogetherImageGenProcessor"
        },
        "llm": {
            "base_url": "https://api.together.xyz/v1",
            "language": "en",
            "messages": [
                {}
            ],
            "model": "Qwen/Qwen2-72B-Instruct",
            "tag": "openai_llm_processor"
        },
        "tts": {
            "args": {
                "gender": "Male",
                "language": "zh",
                "voice_name": "zh-CN-YunjianNeural"
            },
            "tag": "tts_edge"
        }
    },
    "config_list": [],
    "room_manager": {
        "args": {
            "privacy": "public"
        },
        "tag": "daily_room"
    },
    "room_name": "chat-room",
    "room_url": "",
    "services": {
        "img-gen": "togethe

In [18]:
!cat /content/daily_sd_month_narration_bot.json

{
  "chat_bot_name": "DailyMonthNarrationBot",
  "room_name": "chat-room",
  "room_url": "",
  "token": "",
  "room_manager": {
    "tag": "daily_room",
    "args": {
      "privacy": "public"
    }
  },
  "services": {
    "pipeline": "achatbot",
    "llm": "together",
    "img-gen": "together",
    "tts": "edge"
  },
  "config": {
    "llm": {
      "tag": "openai_llm_processor",
      "base_url": "https://api.together.xyz/v1",
      "model": "Qwen/Qwen2-72B-Instruct",
      "language": "en",
      "messages": [{}]
    },
    "img_gen": {
      "tag": "HFStableDiffusionImageGenProcessor",
      "args": {
        "guidance_scale":4.5,
        "device":"auto",
        "is_quantizing":true,
        "steps":28,
        "model": "./models/stabilityai/stable-diffusion-3.5-large"
      }
    },
    "tts": {
      "tag": "tts_edge",
      "args": {
        "voice_name": "zh-CN-YunjianNeural",
        "language": "zh",
        "gender": "Male"
      }
    }
  },
  "config_list": []
}


In [20]:
!DAILY_API_KEY=$DAILY_API_KEY TOGETHER_API_KEY=$TOGETHER_API_KEY HF_API_KEY=$HF_TOKEN \
  python -m src.cmd.bots.main -f /content/daily_sd_month_narration_bot.json

{
    "chat_bot_name": "DailyMonthNarrationBot",
    "config": {
        "img_gen": {
            "args": {
                "device": "auto",
                "guidance_scale": 4.5,
                "is_quantizing": true,
                "model": "./models/stabilityai/stable-diffusion-3.5-large",
                "steps": 28
            },
            "tag": "HFStableDiffusionImageGenProcessor"
        },
        "llm": {
            "base_url": "https://api.together.xyz/v1",
            "language": "en",
            "messages": [
                {}
            ],
            "model": "Qwen/Qwen2-72B-Instruct",
            "tag": "openai_llm_processor"
        },
        "tts": {
            "args": {
                "gender": "Male",
                "language": "zh",
                "voice_name": "zh-CN-YunjianNeural"
            },
            "tag": "tts_edge"
        }
    },
    "config_list": [],
    "room_manager": {
        "args": {
            "privacy": "public"
        },
    

In [ ]:
from google.colab import userdata
REDIS_PASSWORD=userdata.get('REDIS_PASSWORD')


In [ ]:
!LOG_LEVEL=debug REDIS_PASSWORD=$REDIS_PASSWORD python -m src.cmd.bots.main -f /content/task_bot.json